# Vector adder implementation on PYNQ-Z2 Board

In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np

ol = Overlay('/home/xilinx/jupyter_notebooks/lab01_vadd/overlays/vadd.bit')
ol.ip_dict

{'vadd_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb03015d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'vadd_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 's_axi_CONTROL_BUS',
  'parameters': {'C_M_AXI_A_PORT_ADDR_WIDTH': '32',
   'C_M_AXI_A_PORT_ARUSER_WIDTH': '1',
   'C_M_AXI_A_PORT_AWUSER_WIDTH': '1',
   'C_M_AXI_A_PORT_BUSER_WIDTH': '1',
   'C_M_AXI_A_PORT_CACHE_VALUE': '"0011"',
   'C_M_AXI_A_PORT_DATA_WIDTH': '32',
   'C_M_AXI_A_PORT_ENABLE_ID_PORTS': 'false',
   'C_M_AXI_A_PORT_ENABLE_USER_PORTS': 'false',
   'C_M_AXI_A_PORT_ID_WIDTH': '1',
   'C_M_AXI_A_PORT_PROT_VALUE': '"000"',
   'C_M_AXI_A_PORT_RUSER_WIDTH': '1',
   'C_M_AXI_A_PORT_USER_VALUE': '0x00000000',
   'C_M_AXI_A_PORT_WUSER_WIDTH': '1',
   'C_M_AXI_B_PORT_ADDR_WIDTH': '32',
   'C_M_AXI_B_PORT_ARUSER_WIDTH': '1',
   'C_M_AXI_B_PORT_AWUSER_WIDTH': '1',
   'C_M_AXI_B_PORT_BUSER_WIDTH': '1',
   'C_M_AXI_B_PORT_CACHE_VALUE': '"0011"',
   'C_M_AXI_B_PORT_DATA_WIDTH': '32',
  

In [2]:
# ip's key check
ol.ip_dict.keys()

dict_keys(['vadd_0'])

In [3]:
# vadd setting
vadd_ip = ol.vadd_0

In [4]:
'''
HW address
Can not check in jupyter
location : (vitis project) 'solution1/impl/misc/drivers/vadd_v1_0/src/xvadd_hw.h'

// CONTROL_BUS
// 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/COH)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read/COR)
//        bit 7  - auto_restart (Read/Write)
//        others - reserved

// 0x10 : Data signal of a
//        bit 31~0 - a[31:0] (Read/Write)
// 0x18 : Data signal of b
//        bit 31~0 - b[31:0] (Read/Write)

// 0x20 : Data signal of c
//        bit 31~0 - c[31:0] (Read/Write)
'''

"\nHW address\nCan not check in jupyter\nlocation : (vitis project) 'solution1/impl/misc/drivers/vadd_v1_0/src/xvadd_hw.h'\n\n// CONTROL_BUS\n// 0x00 : Control signals\n//        bit 0  - ap_start (Read/Write/COH)\n//        bit 1  - ap_done (Read/COR)\n//        bit 2  - ap_idle (Read)\n//        bit 3  - ap_ready (Read/COR)\n//        bit 7  - auto_restart (Read/Write)\n//        others - reserved\n\n// 0x10 : Data signal of a\n//        bit 31~0 - a[31:0] (Read/Write)\n// 0x18 : Data signal of b\n//        bit 31~0 - b[31:0] (Read/Write)\n\n// 0x20 : Data signal of c\n//        bit 31~0 - c[31:0] (Read/Write)\n"

In [5]:
# predefined value
VECTOR_LEN = 1024

# allocate the physical memory space
input_a = allocate(shape = (VECTOR_LEN), dtype=np.float32)
input_b = allocate(shape = (VECTOR_LEN), dtype=np.float32)
output_c= allocate(shape = (VECTOR_LEN), dtype=np.float32)

In [6]:
# initialize the input and output
a = np.random.rand(VECTOR_LEN)
b = np.random.rand(VECTOR_LEN)
c = np.zeros(VECTOR_LEN)

In [7]:
# write the input
input_a[:] = a[:]
input_b[:] = b[:]
output_c[:] = c[:]

In [8]:
def diff(vector_a, vector_b):
    correct = True
    assert vector_a.size == vector_b.size, 'args shape are must same'
    for a, b in  zip(vector_a, vector_b):
        if np.abs(a - b) > 1e-3:
            correct = False
    if correct :
        print('Inputs are same')
    else :
        print('Inputs are not same')

diff(input_a, a)
diff(input_b, b)

Inputs are same
Inputs are same


In [9]:
# check the allocated address
print(input_a.device_address)
print(input_b.device_address)
print(output_c.device_address)

402952192
402956288
402960384


In [10]:
# write the data to FPGA
vadd_ip.write(0x10, input_a.device_address)
vadd_ip.write(0x18, input_b.device_address)
vadd_ip.write(0x20, output_c.device_address)

In [11]:
# ap starts
vadd_ip.write(0x00,1)

In [12]:
# referenc result
c_ref = a + b

# check the ouptut result
diff(c_ref, output_c)

Inputs are same
